In [ ]:
import os
import pandas as pd
from PyPDF2 import PdfReader

def extract_pdf_values(folder_path, output_csv_path, parse_numbers=False):
    combined_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".PDF") or filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            file_name_without_extension = os.path.splitext(filename)[0]

            # Initialize variables to store isotopes' values
            pb210 = Bi214 = Pb214 = ptsrc_pb210 = None

            # Read in the PDF file
            try:
                reader = PdfReader(file_path)
                if len(reader.pages) < 3:
                    print(f"PDF file '{filename}' has less than 3 pages. Skipping.")
                    continue
                page = reader.pages[2]
                text = page.extract_text()
                lines = text.split('\n')

                # Check if the filename starts with 'PtSrc_'
                if filename.startswith("PtSrc_"):
                    # Only extract Pb-210 value for PtSrc files
                    for line in lines:
                        if 'Pb-210' in line:
                            ptsrc_pb210, PtSrc_Pb210error = line.split()[-2:]
                            break  # Exit the loop after finding Pb-210
                    # Store only Pb-210 values if found
                    if ptsrc_pb210 is not None and PtSrc_Pb210error is not None:
                        data = {'File': filename, 'Pb-210': float(ptsrc_pb210), 'Pb-210 error': float(PtSrc_Pb210error)}
                    else:
                        print(f"Pb-210 not found in '{filename}'. Skipping.")
                        continue
                else:
                    # Extract values for other files
                    for line in lines:
                        if 'Pb-210' in line:
                            pb210, pb210error = line.split()[-2:]
                        elif 'Bi-214' in line:
                            Bi214, Bi214error = line.split()[-2:]
                        elif 'Pb-214' in line:
                            Pb214, Pb214error = line.split()[-2:]

                    # Store values only if they were found
                    if pb210 is not None and pb210error is not None:
                        data = {
                            'File': filename,
                            'Pb-210': float(pb210),
                            'Pb-210 error': float(pb210error),
                            'Bi-214': float(Bi214) if Bi214 else None,
                            'Bi-214 error': float(Bi214error) if Bi214error else None,
                            'Pb-214': float(Pb214) if Pb214 else None,
                            'Pb-214 error': float(Pb214error) if Pb214error else None
                        }
                    else:
                        print(f"Pb-210 not found in '{filename}'. Skipping.")
                        continue

                combined_data.append(data)
            except Exception as e:
                print(f"Error processing PDF file '{filename}': {e}")

    combined_df = pd.DataFrame(combined_data)

    if parse_numbers:
        combined_df[['Pb-210', 'Bi-214', 'Pb-214']] = combined_df[['Pb-210', 'Bi-214', 'Pb-214']].apply(lambda x: x.str.strip().astype(float))

    combined_df.to_csv(output_csv_path, index=False)

# Example usage (mac users):
#If you are using Windows, add an "r" before the "" of the pathname
extract_pdf_values("folderpath", "corename_canberradata.csv")



